In [1]:
import torch.nn as nn
import torch

In [2]:
B, N, d = 10, 128, 256
d_k, d_v = 300, 320
h = 8

X = torch.ones(size=(B, N, d))
W_Q = torch.ones(size=(h, d, d_k))
W_K = torch.ones(size=(h, d, d_k))
W_V = torch.ones(size=(h, d, d_v))

Q = torch.einsum('bik,hkj->bhij', X, W_Q)
print(Q.size())

K = torch.einsum('bik,hkj->bhij', X, W_K)
print(K.size())

V = torch.einsum('bik,hkj->bhij', X, W_V)
print(V.size())

torch.Size([10, 8, 128, 300])
torch.Size([10, 8, 128, 300])
torch.Size([10, 8, 128, 320])


In [3]:
QKT = torch.einsum('bhik,bhkj->bhij', Q, torch.transpose(K, 2, 3))
print(QKT.size())

torch.Size([10, 8, 128, 128])


In [4]:
sm = nn.Softmax(dim=2)
A = sm(QKT)

print(A[0,0,:,:].sum(dim=1))    # they're indeed probability distributions

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])


In [5]:
V = torch.einsum('bik,hkj->bhij', X, W_V)
print(V.size())

torch.Size([10, 8, 128, 320])


In [6]:
AV = torch.einsum('bhik,bhkj->bhij', A, V)
print(AV.size())

torch.Size([10, 8, 128, 320])


In [10]:
AV_concat = torch.reshape(AV, shape=(B, N, h*d_v))
print(AV_concat.size())

torch.Size([10, 128, 2560])


In [11]:
W_O = torch.ones(size=(h*d_v, d))
SA_out = torch.einsum('bni,id->bnd', AV_concat, W_O)
print(SA_out.size())

torch.Size([10, 128, 256])


## Todo : 

- Define batch einsum operations on layer norm
- Same on feedforward
- Implement a Transformer block using that.